In [43]:
import sys
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mping
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import transforms

from data_loader import get_loader
from model import ClassifiCNN

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

### set high parameters

In [44]:
batch_size = 32
num_epochs = 1
class_size = 61


# 图片格式转化
transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.RandomCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

### init model

In [45]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

data_loader = get_loader(transform_train, mode='test', batch_size=batch_size)

# set the total number of training steps per epoch
total_step = int(len(data_loader.dataset)/batch_size)

classify_model = ClassifiCNN(class_size)

classify_model.to(device)

criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(classify_model.fc.parameters())

/Users/Young/anaconda3/lib/python3.6/site-packages/torchvision-0.2.1-py3.6.egg/torchvision/models/densenet.py:212: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data)


### time to train model

In [14]:
for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        images, targets = next(iter(data_loader))
        
        images = images.to(device)
        targets = targets.type(torch.LongTensor).to(device)
        
        classify_model.zero_grad()
        
        outputs = classify_model(images)
        
        
        loss = criterion(outputs, targets.view(-1))
        
        loss.backward()
        
        optimizer.step()
        
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f' % (epoch, num_epochs, i_step, total_step, loss.item())
        
        print('\r' + stats, end='')
        sys.stdout.flush()
        
    torch.save(classify_model.fc.state_dict(), os.path.join('./models', 'class_model_%d.pkl' % epoch))

Epoch [1/1], Step [155/155], Loss: 1.1004

### How good is the model?

In [46]:
df_refer = data_loader.dataset.refer

In [59]:
img, _ = data_loader.dataset[101]
img = img.unsqueeze(0).to(device)
classify_model(img).argmax().item()

11

In [38]:
predict = []
for img, _ in data_loader.dataset:
    img = img.unsqueeze(0).to(device)
    p = classify_model(img).argmax().item()
    predict.append(p)